In [1]:
import sys,os
sys.path.append("../")
import gensim
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText as FT_gensim
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pickle



In [2]:
def get_word_vecs(wiki_vec, masks, tokens, token_string_df):
    if tokens and tokens[-1] == -1:
        tokens = tokens[:-1]
    if tokens and tokens[-1] == -2:
        tokens = tokens[:-1]
    if len(tokens) == 0:
        return np.zeros( wiki_vec.vector_size)
    tokens = token_string_df[np.array(tokens)].values
    tokens_in_vocab_mask = masks.loc[tokens, "mask"].values
#     print(tokens_in_vocab_mask)
    in_vocab_tokens = tokens[tokens_in_vocab_mask]
    if np.any(tokens_in_vocab_mask):
        return np.average(wiki_vec[in_vocab_tokens], axis=0)


    else:
        return np.zeros( wiki_vec.vector_size)


In [3]:
%%time 
wiki_vec = KeyedVectors.load_word2vec_format('../../wordvectors/wiki.en.vec', binary=False, limit=1000000)

CPU times: user 10min 46s, sys: 44.4 s, total: 11min 31s
Wall time: 12min 27s


In [4]:
vocab_list = list(wiki_vec.vocab)
filtered_vocab = [ t for t in vocab_list[20:] if len(t) > 3]

display(f"lenght of vocabulary is {len(vocab_list)} words")
display(f"lenght of vocabulary is {len(filtered_vocab)} words")

vocab_list = np.array(vocab_list)
filtered_vocab = np.array(filtered_vocab)

'lenght of vocabulary is 1000000 words'

'lenght of vocabulary is 970137 words'

### reading the change object and clustering.

In [17]:
%%time

article_name = "Berlin_Wall"
change_object_dir =  "../data/change objects/"

change_object_file_name = f"{article_name}_vec.npz"
filename =  f"{article_name}_change.h5"
# change_file_name = f"{article_name}.pkl"
change_object_file = os.path.join(change_object_dir, filename)


if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
display(change_object_dataframe.shape)

(19641, 12)

CPU times: user 2.8 s, sys: 648 ms, total: 3.44 s
Wall time: 3.44 s


In [18]:
%%time
content_dir = "../data/content/"
filename = article_name + ".h5"
filepath = os.path.join(content_dir, filename)
with pd.HDFStore(filepath, 'r') as store:
    all_rev = store.get("all_tokens")
unique_str = np.unique(all_rev.str)
str_in_filtered_vocab_mask = np.isin(unique_str, filtered_vocab, assume_unique=True)

filtered_vocab_masks_df = pd.DataFrame({ "str":unique_str, "mask":str_in_filtered_vocab_mask}).set_index("str")



token_string_df = all_rev.set_index("token_id")["str"]

CPU times: user 10min 1s, sys: 200 ms, total: 10min 1s
Wall time: 10min 1s


## Make Vector from change object.

In [19]:

left_neighbour_matrix = np.stack(change_object_dataframe["left_token"].apply(lambda token_set: token_set[::-1][:10] ).apply(lambda token_set: get_word_vecs(wiki_vec, filtered_vocab_masks_df, token_set, token_string_df)).values)
right_neighbour_matrix = np.stack(change_object_dataframe["right_token"].apply(lambda token_set: token_set[:10] ).apply(lambda token_set: get_word_vecs(wiki_vec, filtered_vocab_masks_df, token_set, token_string_df)).values)
filtered_neighbour10_matrix = np.c_[ left_neighbour_matrix, right_neighbour_matrix]

left_neighbour_matrix = np.stack(change_object_dataframe["left_token"].apply(lambda token_set: token_set[::-1][:4] ).apply(lambda token_set: get_word_vecs(wiki_vec, filtered_vocab_masks_df, token_set, token_string_df)).values)
right_neighbour_matrix = np.stack(change_object_dataframe["right_token"].apply(lambda token_set: token_set[:4] ).apply(lambda token_set: get_word_vecs(wiki_vec, filtered_vocab_masks_df, token_set, token_string_df)).values)
filtered_neighbour4_matrix = np.c_[ left_neighbour_matrix, right_neighbour_matrix]

## Saving change object vector  to file

In [20]:
change_vector_dir = "../data/change_vector/"
change_vec_filename = f"{article_name}.npz"
change_vector_file = os.path.join(change_vector_dir, change_vec_filename)

arrays_to_save = {

    "4_clean_not_weighted": filtered_neighbour4_matrix,
    "10_clean_not_weighted": filtered_neighbour10_matrix
}
with open(change_vector_file, "wb") as file:
    np.savez(file, **arrays_to_save)